In [ ]:
import pandas as pd

def imputar_categoricas(df, columnas, umbral_nulos_alto=0.20, umbral_nulos_bajo=0.05,
                        umbral_moda_bajo=0.50, umbral_moda_medio=0.60, umbral_ventaja=0.20,
                        etiqueta_unknown="Unknown"):
    """
    Imputa nulos en variables categóricas siguiendo reglas simples y justificables en EDA.

    Reglas (resumen):
    1) Si el % de nulos es ALTO (> umbral_nulos_alto, por defecto 20%) → imputar con etiqueta_unknown.
       - Motivo: imputar por moda con muchos nulos puede inventar demasiada información.

    2) Si el % de nulos es BAJO (<= umbral_nulos_bajo, por defecto 5%) o MEDIO (entre 5% y 20%):
       - Solo imputamos con la MODA si hay una categoría realmente dominante.
       - Para considerar "dominante" exigimos 2 condiciones:
         a) La moda supera un umbral según el % de nulos:
            - Si nulos BAJOS: moda >= umbral_moda_bajo (50% por defecto)
            - Si nulos MEDIOS: moda >= umbral_moda_medio (60% por defecto)
         b) La moda tiene suficiente ventaja sobre la 2ª categoría:
            - (pct_moda - pct_segunda) >= umbral_ventaja (20 puntos porcentuales por defecto)

       Si no se cumple lo anterior → imputar con etiqueta_unknown.

    Parámetros
    ----------
    df : pandas.DataFrame
        DataFrame de entrada (se modifica y también se devuelve).
    columnas : list[str]
        Lista de columnas categóricas a imputar.
    umbral_nulos_alto : float, default 0.20
        Por encima de este porcentaje de nulos se usa etiqueta_unknown.
    umbral_nulos_bajo : float, default 0.05
        Hasta este porcentaje de nulos se considera "bajo".
    umbral_moda_bajo : float, default 0.50
        Umbral mínimo de la moda si los nulos son bajos.
    umbral_moda_medio : float, default 0.60
        Umbral mínimo de la moda si los nulos son medios (5%-20%).
    umbral_ventaja : float, default 0.20
        Ventaja mínima (en proporción, 0.20 = 20 puntos porcentuales) entre la moda y la 2ª categoría.
    etiqueta_unknown : str, default "Unknown"
        Etiqueta para imputar cuando no se quiere usar la moda.

    Returns
    -------
    pandas.DataFrame
        El mismo DataFrame con los nulos imputados en las columnas indicadas.
    """

    total = len(df)

    for col in columnas:
        print(f"\n📌 Analizando columna: {col}")
        
        # Si la columna no existe, evitamos error y seguimos
        if col not in df.columns:
            print(f"❌ La columna {col} no existe en el DataFrame. Se omite.")
            continue

        nulos = df[col].isnull().sum()
        porcentaje_nulos = nulos / total if total > 0 else 0

        print(f"   → Nulos: {nulos} de {total} ({porcentaje_nulos:.2%})")

        # Caso 1: muchos nulos -> Unknown directamente
        if porcentaje_nulos > umbral_nulos_alto:
            print(
                f"   🔴 Porcentaje de nulos > {umbral_nulos_alto:.0%} "
                f"→ se crea la categoría '{etiqueta_unknown}'"
            )
            df[col] = df[col].fillna(etiqueta_unknown)
            continue
        
        # Calculamos frecuencias (sin nulos) para decidir si hay moda dominante
        valores = df[col].value_counts(dropna=True)

        if len(valores) == 0:
            # No hay valores no nulos para decidir moda (columna vacía o todo nulo)
            print(
                f"   🔴 No hay valores no nulos para decidir moda "
                f"→ se crea la categoría '{etiqueta_unknown}'"
            )
            df[col] = df[col].fillna(etiqueta_unknown)
            continue

        # Frecuencia y porcentaje de la moda
        primero = valores.iloc[0]
        pct_primero = primero / total # proporción sobre el total de filas

        # Frecuencia y porcentaje de la segunda categoría (si existe)
        if len(valores) > 1:
            segundo = valores.iloc[1]
            pct_segundo = segundo / total
        else:
            pct_segundo = 0.0

        ventaja = pct_primero - pct_segundo # ventaja de la moda frente a la 2ª (en proporción)

        print(f"   → Moda: {valores.index[0]} ({pct_primero:.2%})")
        print(f"   → 2ª categoría: {valores.index[1] if len(valores) > 1 else 'No existe'} ({pct_segundo:.2%})")
        print(f"   → Ventaja de la moda: {ventaja:.2%}")

        # Umbral de moda depende de si el % de nulos es bajo o medio
        if porcentaje_nulos <= umbral_nulos_bajo:
            umbral_moda = umbral_moda_bajo
            print(
                f"   → Nulos bajos (≤ {umbral_nulos_bajo:.0%}), "
                f"umbral de moda requerido: {umbral_moda:.0%}"
            )
        else:
            umbral_moda = umbral_moda_medio
            print(
                f"   → Nulos medios (> {umbral_nulos_bajo:.0%} y ≤ {umbral_nulos_alto:.0%}), "
                f"umbral de moda requerido: {umbral_moda:.0%}"
            )
        
        # Caso 2: imputar por moda solo si es dominante y con ventaja suficiente
        if (pct_primero >= umbral_moda) and (ventaja >= umbral_ventaja):
            moda = df[col].mode(dropna=True)[0]
            print(
                f"   🟢 La moda es dominante y con ventaja suficiente "
                f"(≥ {umbral_ventaja:.0%}) → se imputan nulos con '{moda}'"
            )
            df[col] = df[col].fillna(moda)
        else:
            print(
                f"   🟡 La moda NO es lo suficientemente dominante "
                f"→ se crea la categoría '{etiqueta_unknown}'"
            )
            df[col] = df[col].fillna(etiqueta_unknown)

    return df


In [ ]:
cols_cat = df.select_dtypes(include=["object", "category"]).columns.tolist()
cols_cat = [c for c in cols_cat if c.lower() != "attrition"]  # excluimos attrition

# normaliza falsos nulos solo en esas columnas
df[cols_cat] = df[cols_cat].replace(r'^\s*$', pd.NA, regex=True)

# imputación
df = imputar_categoricas(df, cols_cat)

In [ ]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler


def imputar_numericas(df, columnas, umbral_nulos_bajo=0.05, umbral_nulos_alto=0.20,
                      n_neighbors=5, crear_indicador_missing=True, usar_knn_en_alto=False):
    """
    Imputa nulos en variables numéricas siguiendo reglas simples y justificables en EDA,
    usando mediana (robusta) cuando el % de nulos es bajo y KNNImputer cuando es moderado.

    Reglas (resumen):
    1) Si el % de nulos es BAJO (<= umbral_nulos_bajo, por defecto 5%) → imputar con mediana.
       - Motivo: con pocos nulos, la mediana es estable y minimiza el efecto de outliers.

    2) Si el % de nulos es MODERADO (> umbral_nulos_bajo y <= umbral_nulos_alto, por defecto 5%-20%)
       → imputar con KNNImputer (por defecto k=5) usando registros similares.
       - Motivo: con más nulos, KNN puede aprovechar el “contexto” de otras variables numéricas.

    3) Si el % de nulos es ALTO (> umbral_nulos_alto, por defecto 20%):
       - (Opcional) crear una variable indicadora de missingness: col + "_missing"
       - Imputar con mediana por defecto (más estable). Si quieres, puedes activar KNN también en alto
         con usar_knn_en_alto=True.

    IMPORTANTE SOBRE KNN + ESCALADO:
    KNN funciona con distancias entre filas. Si las columnas numéricas están en escalas distintas
    (por ejemplo, una en 0-10 y otra en 0-10.000), la columna de rango grande dominaría la distancia.
    Por eso hacemos:
      - Escalado (StandardScaler) → KNNImputer → desescalado
    Así todas las columnas “pesan” parecido al calcular similitud.

    Parámetros
    ----------
    df : pandas.DataFrame
        DataFrame de entrada (se modifica y también se devuelve).
    columnas : list[str]
        Lista de columnas numéricas a imputar.
    umbral_nulos_bajo : float, default 0.05
        Hasta este porcentaje de nulos se considera "bajo".
    umbral_nulos_alto : float, default 0.20
        Por encima de este porcentaje de nulos se considera "alto".
    n_neighbors : int, default 5
        Número de vecinos (k) para KNNImputer.
    crear_indicador_missing : bool, default True
        Si True, cuando el % de nulos es alto se crea una columna col+"_missing" (0/1).
    usar_knn_en_alto : bool, default False
        Si True, en % de nulos alto también se usa KNN (con escalado). Si False, se usa mediana.

    Returns
    -------
    pandas.DataFrame
        El mismo DataFrame con los nulos imputados en las columnas indicadas.
    """

    total = len(df)

    # Nos aseguramos de trabajar solo con columnas que existen
    columnas_validas = [c for c in columnas if c in df.columns]
    columnas_no_encontradas = [c for c in columnas if c not in df.columns]
    for c in columnas_no_encontradas:
        print(f"{c} no existe en el DataFrame. Se omite.")

    # También nos aseguramos de que sean numéricas (por si te cuelas en la lista)
    cols_num_df = df.select_dtypes(include=[np.number]).columns.tolist()
    columnas_validas = [c for c in columnas_validas if c in cols_num_df]

    # Si no queda ninguna, salimos sin romper nada
    if len(columnas_validas) == 0:
        print("No hay columnas numéricas válidas para imputar.")
        return df

    # Preparamos imputador de mediana (lo reutilizamos)
    imputer_mediana = SimpleImputer(strategy="median")

    # Para KNN con escalado, necesitamos un “bloque” numérico:
    # Usamos TODAS las numéricas del DF, porque KNN se beneficia de más contexto.
    # (Esto NO significa que imputemos todas: solo guardamos de vuelta las columnas objetivo.)
    cols_num_contexto = cols_num_df

    for col in columnas_validas:
        print(f"\n📌 Analizando columna numérica: {col}")

        nulos = df[col].isnull().sum()
        porcentaje_nulos = nulos / total if total > 0 else 0
        print(f"   → Nulos: {nulos} de {total} ({porcentaje_nulos:.2%})")

        # CASO 1: % de nulos bajo -> mediana
        if porcentaje_nulos <= umbral_nulos_bajo:
            print(
                f"   🟢 Nulos bajos (≤ {umbral_nulos_bajo:.0%}) "
                f"→ imputación con MEDIANA (SimpleImputer)"
            )
            df[[col]] = imputer_mediana.fit_transform(df[[col]])
            continue

        # CASO 2: % de nulos moderado -> KNN con escalado
        if porcentaje_nulos <= umbral_nulos_alto:
            print(
                f"   🟡 Nulos moderados (> {umbral_nulos_bajo:.0%} y ≤ {umbral_nulos_alto:.0%}) "
                f"→ imputación con KNN (k={n_neighbors}) + ESCALADO"
            )

            # 1) Cogemos el bloque numérico completo (contexto)
            X = df[cols_num_contexto].copy()

            # 2) Escalamos (standardization): (x - media) / desviación
            #    Esto hace comparables las columnas para calcular distancias.
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X)

            # 3) Imputamos en el espacio escalado
            knn = KNNImputer(n_neighbors=n_neighbors)
            X_imputed_scaled = knn.fit_transform(X_scaled)

            # 4) Desescalamos para volver a las unidades originales
            X_imputed = scaler.inverse_transform(X_imputed_scaled)

            # 5) Volvemos a DataFrame para poder asignar por columnas
            X_imputed = pd.DataFrame(X_imputed, columns=cols_num_contexto, index=df.index)

            # 6) Solo guardamos la columna objetivo (para no tocar otras numéricas fuera de tu lista)
            df[col] = X_imputed[col]

            print(f"   ✅ {col} imputada con KNN + escalado (solo se asigna esta columna).")
            continue

        # CASO 3: % de nulos alto
        print(
            f"   🔴 Nulos altos (> {umbral_nulos_alto:.0%}) "
            f"→ se considera missingness + imputación robusta"
        )

        if crear_indicador_missing:
            indicador = f"{col}_missing"
            # 1 si era nulo, 0 si no
            df[indicador] = df[col].isnull().astype(int)
            print(f"   → Se crea indicador de missingness: {indicador} (1=nulo, 0=no nulo)")

        if usar_knn_en_alto:
            print(
                f"   🟠 usar_knn_en_alto=True "
                f"→ imputación con KNN (k={n_neighbors}) + ESCALADO"
            )

            X = df[cols_num_contexto].copy()
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X)

            knn = KNNImputer(n_neighbors=n_neighbors)
            X_imputed_scaled = knn.fit_transform(X_scaled)

            X_imputed = scaler.inverse_transform(X_imputed_scaled)
            X_imputed = pd.DataFrame(X_imputed, columns=cols_num_contexto, index=df.index)

            df[col] = X_imputed[col]
            print(f"   ✅ {col} imputada con KNN + escalado (solo se asigna esta columna).")

        else:
            print("   🟠 Se imputa con MEDIANA (SimpleImputer) por estabilidad.")
            df[[col]] = imputer_mediana.fit_transform(df[[col]])

    return df


---
## **10. FUNCIÓN DE LIMPIEZA GENERAL**

Esta función integra todos los pasos anteriores en un solo proceso automatizado.
Es útil cuando quieres aplicar todo el flujo de limpieza de una sola vez.

In [ ]:
def limpieza_general(
    df,
    id_columna='employee_number',
    mapeo_tipos=None,
    mapeos_texto=None,
    mapeos_ordinales=None,
    columnas_categoricas_nulos=None,
    columnas_numericas_nulos=None,
    mostrar_resumen=True
):
    """
    Ejecuta el proceso completo de limpieza de datos en un DataFrame.
    
    Esta función orquesta todos los pasos de limpieza en el orden correcto:
    1. Normalización de nombres de columnas (snake_case)
    2. Establecer columna ID como índice
    3. Eliminación de filas duplicadas
    4. Eliminación de columnas sin aporte analítico
    5. Conversión de tipos de datos
    6. Normalización de columnas de texto
    7. Mapeo de columnas ordinales
    8. Imputación de nulos en categóricas
    9. Imputación de nulos en numéricas

    Parámetros:
    -----------
    df : pd.DataFrame
        DataFrame original a limpiar
    id_columna : str, default='employee_number'
        Nombre de la columna que se usará como índice
    mapeo_tipos : dict, optional
        Diccionario de conversión de tipos {'columna': tipo}
    mapeos_texto : dict, optional
        Diccionario de reemplazos en texto {'columna': {'viejo': 'nuevo'}}
    mapeos_ordinales : dict, optional
        Diccionario de mapeos ordinales {'columna': {1: 'etiqueta'}}
    columnas_categoricas_nulos : list, optional
        Lista de columnas categóricas donde imputar nulos
    columnas_numericas_nulos : list, optional
        Lista de columnas numéricas donde imputar nulos
    mostrar_resumen : bool, default=True
        Si True, imprime resumen de cada paso

    Retorna:
    --------
    pd.DataFrame
        DataFrame limpio y listo para análisis
    """

    # Crear copia para no modificar el DataFrame original
    df = df.copy()

    # PASO 1: Normalizar nombres de columnas
    df.columns = normalizar_nombres_columnas(df.columns.tolist(), mostrar_resumen=mostrar_resumen)

    # PASO 2: Establecer ID como índice
    df = usar_columna_como_indice(df, columna_original=id_columna, indice='id')

    # PASO 3: Eliminar duplicados
    df = eliminar_filas_duplicadas(df)

    # PASO 4: Eliminar columnas sin aporte analítico
    df = eliminar_columnas_sin_aporte_analitico(df)

    # PASO 5: Conversión de tipos (si se proporcionó mapeo)
    if mapeo_tipos:
        df = convertir_tipos_columnas(df, mapeo_tipos, mostrar_resumen=mostrar_resumen)

    # PASO 6: Normalización de texto (si se proporcionaron mapeos)
    if mapeos_texto:
        df = normalizar_columnas_texto(df, mapeos_reemplazo=mapeos_texto, mostrar_resumen=mostrar_resumen)

    # PASO 7: Mapeo de ordinales (si se proporcionaron mapeos)
    if mapeos_ordinales:
        df = mapear_columnas_ordinales(df, mapeos_ordinales, mostrar_resumen=mostrar_resumen)

    # PASO 8: Imputación de categóricas (si se especificaron columnas)
    if columnas_categoricas_nulos:
        df = imputar_categoricas(df, columnas_categoricas_nulos)

    # PASO 9: Imputación de numéricas (si se especificaron columnas)
    if columnas_numericas_nulos:
        df = imputar_numericas(df, columnas_numericas_nulos)

    if mostrar_resumen:
        print("\n🟢 Limpieza general completada.")

    return df